In [1]:
#standard ds imports
import pandas as pd
import numpy as np
#viz and stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# .py imports
import wrangle_g as wr
#import modeling as md
import os
import nltk
import unicodedata
import re
#sklearn imports
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#CATboost imports
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('lang_data_prepped.csv')

In [3]:
df

,repo,language,readme_contents,stopped
0,ujjwalkarn/DataScienceR,R,# R Data Science Tutorials\n- This repo contai...,data science tutorials repo contain curated li...
1,IndrajeetPatil/ggstatsplot,R,---\noutput: github_document\n---\n\n <!-- RE...,output githubdocument readmemd generate readme...
2,easystats/easystats,R,"---\noutput: github_document\n---\n\n```{r, wa...",output githubdocument warningfalse messagefals...
3,seandavi/sars2pack,R,---\noutput: \n - rmarkdown::github_document\...,output rmarkdowngithubdocument rmarkdownhtmldo...
4,markvanderloo/lumberjack,R,## Track changes in data\n[![Build Status](htt...,track change data build statushttpstravisciorg...
...,...,...,...,...
286,rahulbordoloi/Machine-Learning,Python,# Machine-Learning\nContains my Glossary for M...,machinelearning contain glossary machine learn...
287,Oslandia/QDeeplandia,Python,QGIS plugin dedicated to 2D semantic segmentat...,qgis plugin dedicate 2d semantic segmentation ...
288,arjunbhasin2013/BaselinePredictionMLFramework,Python,# Baseline Prediction Machine Learning Framewo...,baseline prediction machine learn framework ma...
289,PonderaLab/datascience4economists,Python,"# Datascience4economists\n\nFirst of all, welc...",datascience4economists first welcome repositor...


In [4]:
df = pd.get_dummies(df, columns=['language'], drop_first=True)

In [5]:
df.head()

,repo,readme_contents,stopped,language_R
0,ujjwalkarn/DataScienceR,# R Data Science Tutorials\n- This repo contai...,data science tutorials repo contain curated li...,1
1,IndrajeetPatil/ggstatsplot,---\noutput: github_document\n---\n\n <!-- RE...,output githubdocument readmemd generate readme...,1
2,easystats/easystats,"---\noutput: github_document\n---\n\n```{r, wa...",output githubdocument warningfalse messagefals...,1
3,seandavi/sars2pack,---\noutput: \n - rmarkdown::github_document\...,output rmarkdowngithubdocument rmarkdownhtmldo...,1
4,markvanderloo/lumberjack,## Track changes in data\n[![Build Status](htt...,track change data build statushttpstravisciorg...,1


## BASELINE-using all datapoints

In [6]:
df.language_R.value_counts()

0    167
1    124
Name: language_R, dtype: int64

In [7]:
df['baseline'] = 0

In [8]:
baseline_accuracy = (df.baseline == df.language_R).mean()
print(f'baseline accuracy: {baseline_accuracy:.2%}')

baseline accuracy: 57.39%


In [9]:
subset = df[df.language_R == 0]
baseline_recall = (subset.baseline == subset.language_R).mean()
print(f'baseline recall: {baseline_recall:.2%}')

baseline recall: 100.00%


In [10]:
subset = df[df.baseline == 0]
baseline_precision = (subset.baseline == subset.language_R).mean()
print(f'baseline precision: {baseline_precision:.2%}')

baseline precision: 57.39%


In [11]:
df.drop(columns='baseline',inplace=True)

## Train/Validate/Test Split

In [12]:
df.rename(columns = {'language_R':'language'}, inplace = True)

In [13]:
train, validate, test = wr.final_wrangle(df)

In [14]:
train.shape[0],validate.shape[0],test.shape[0]

(162, 70, 59)

## Create a column showing token count

In [15]:
train['token_cnt'] = [len(row['stopped'].split()) for _, row in train.iterrows()]

In [16]:
train.head()

,repo,readme_contents,stopped,language,token_cnt
117,storieswithsiva/US-Job-Market-Analysis,# US Job Analysis in R\n[![Makes people smile]...,us job analysis make people smilehttpsfortheba...,1,381
8,adityashrm21/Bike-Sharing-Demand-Kaggle,# Bike-Sharing-Demand---Kaggle\n\nSolution to ...,bikesharingdemandkaggle solution kaggle knowle...,1,12
63,Sarah-Hesham-2022/Bio-Statistics-R-Coding-On-D...,# Bio-Statistics-R-Coding-On-Datasets-Analysis...,biostatisticsrcodingondatasetsanalysisandplott...,1,35
268,Tech-Matt/game_of_life,# Game of Life\n\n## Premise\nThis is the firs...,game life premise first readme im write projec...,0,125
105,vincenzorusso3/tesi-triennale,# PPMI Research\nThe main goal is to create a ...,ppmi research main goal create system order cl...,1,78


In [17]:
train.rename(columns = {'language':'language_R'}, inplace = True)

In [18]:
train = train.drop(columns=['repo','readme_contents'])

In [19]:
train.head()

,stopped,language_R,token_cnt
117,us job analysis make people smilehttpsfortheba...,1,381
8,bikesharingdemandkaggle solution kaggle knowle...,1,12
63,biostatisticsrcodingondatasetsanalysisandplott...,1,35
268,game life premise first readme im write projec...,0,125
105,ppmi research main goal create system order cl...,1,78


In [21]:
train.language_R.value_counts()

0    93
1    69
Name: language_R, dtype: int64